<a href="https://colab.research.google.com/github/divyachappa12/fmml_modules/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [2]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [3]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [4]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [5]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [6]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [7]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [8]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [9]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [11]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [12]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [13]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [14]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [15]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


Yes, averaging the validation accuracy across multiple splits can provide more consistent and reliable results. Here's why:

1. Reduction of Variability:
When you split your dataset into training and validation sets, the performance can vary depending on how the data is divided. Some splits might be easier or harder for the model, leading to fluctuations in validation accuracy. By averaging the accuracy across multiple splits, you reduce the impact of any particular split that might be unrepresentative or have an unusual distribution.
2. More Representative Estimate:
Averaging across multiple splits gives a more robust estimate of the model’s performance on unseen data. This method accounts for variability in the data, providing a better overall picture of how well the model is likely to perform in practice.
3. Cross-Validation:
This approach is formalized in cross-validation techniques like k-fold cross-validation. In k-fold cross-validation, the dataset is divided into k subsets (folds). The model is trained on k-1 folds and validated on the remaining fold, with this process repeated k times (once for each fold). The final validation accuracy is typically the average of the accuracies from all the folds, providing a more stable and generalizable performance metric.
4. Mitigation of Overfitting:
By using multiple splits, you reduce the chance that the model is overfitting to any particular validation set. This is because the model is exposed to a wider variety of data during training and evaluation, making the average accuracy a better indicator of generalization.
In summary, averaging validation accuracy across multiple splits is a standard and effective way to obtain more consistent and reliable estimates of model performance, especially in scenarios where data variability might affect the results.

Yes, averaging the validation accuracy across multiple splits generally provides a more accurate and reliable estimate of the model's test accuracy compared to using a single validation split. Here's a detailed explanation of why this is the case:

1. Reduction of Variance
Single Split Limitation: When you use a single train-validation split, the estimated validation accuracy can be highly sensitive to how the data is partitioned. If the split happens to include an unrepresentative subset of the data in the validation set, the accuracy estimate may be misleading.

Averaging Multiple Splits: By averaging validation accuracies over multiple splits (as in k-fold cross-validation), you mitigate the impact of any single unrepresentative split. This leads to a more stable and reliable estimate by smoothing out the variability inherent in different data partitions.

2. Comprehensive Utilization of Data
Single Split Limitation: A single validation split only evaluates the model on a subset of the data, potentially ignoring patterns present in other parts of the dataset.

Averaging Multiple Splits: Techniques like k-fold cross-validation ensure that every data point is used for both training and validation across different iterations. This comprehensive usage helps capture a wider range of data patterns, leading to a more accurate reflection of the model's performance on unseen data.

3. Bias-Variance Trade-off
Single Split Limitation: A single validation set might introduce bias if it is not representative of the entire dataset, affecting the generalization estimate.

Averaging Multiple Splits: By using multiple validation sets, cross-validation reduces both bias and variance in the performance estimate. The averaging process ensures that the final estimate is less likely to be skewed by anomalies in any particular split.

4. Better Generalization Estimate
Single Split Limitation: The estimate from a single split might not generalize well to truly unseen data, especially if the split is not representative.

Averaging Multiple Splits: Cross-validation provides a more robust estimate of how the model is expected to perform on new, unseen data by effectively simulating multiple scenarios of train-test splits.

5. Handling Limited Data
Single Split Limitation: In cases of limited data, a single train-validation split might not utilize the data efficiently, leading to less reliable estimates.

Averaging Multiple Splits: Cross-validation maximizes the use of available data by ensuring that each data point is used for both training and validation, which is particularly beneficial when the dataset is small.

6. Empirical Evidence
Numerous studies and practical applications in machine learning have demonstrated that cross-validation often provides more accurate and stable performance estimates compared to single split validation. This empirical evidence supports the theoretical advantages outlined above.
Caveats to Consider
While averaging validation accuracy across multiple splits generally leads to a more accurate estimate of test accuracy, there are some considerations to keep in mind:

Computational Cost: Multiple training and validation runs increase computational requirements, which might be a limitation for very large datasets or complex models.

Data Leakage: Proper cross-validation requires careful handling to ensure that there is no leakage of information between training and validation sets, especially in cases involving data preprocessing steps.

Choice of Splits: The method of splitting (e.g., stratified k-fold for classification tasks) should align with the data characteristics to ensure that each fold is representative.

Conclusion
Averaging validation accuracy across multiple splits, as done in cross-validation techniques, provides a more accurate and reliable estimate of a model's test accuracy compared to relying on a single validation split. This approach leverages multiple data partitions to reduce variability, mitigate bias, and better simulate the model's performance on unseen data, thereby offering a closer approximation to the true generalization performance.


I prefer this response




The number of iterations, particularly in the context of cross-validation or repeated training and validation cycles, can significantly affect the accuracy and stability of the estimate of model performance. Here’s how:

1. Stability of the Estimate:
Higher Number of Iterations: As the number of iterations (or folds in k-fold cross-validation) increases, the estimate of model performance generally becomes more stable. This is because more iterations reduce the impact of any single, potentially unrepresentative, split or fold. The estimate smooths out fluctuations, leading to a more consistent and reliable measure of accuracy.
Lower Number of Iterations: With fewer iterations, the estimate is more susceptible to variability and outliers, as each split or fold has a larger impact on the final averaged result. This can lead to less stable and potentially misleading estimates of accuracy.
2. Accuracy of the Estimate:
Higher Number of Iterations: A greater number of iterations tends to give a more accurate estimate of test accuracy because the model is validated on a larger variety of data subsets. This comprehensive evaluation helps the model better capture the true distribution of the data, leading to an estimate that is closer to the model’s actual performance on unseen data.
Lower Number of Iterations: With fewer iterations, the estimate might be less accurate because the model is exposed to fewer variations in the data. This could result in either an overly optimistic or pessimistic estimate of performance, depending on the specific splits used.
3. Convergence of the Estimate:
Diminishing Returns: While increasing the number of iterations generally improves the estimate’s accuracy and stability, there are diminishing returns after a certain point. Beyond a certain number of iterations, the estimate may converge, meaning that additional iterations only lead to marginal improvements. The optimal number of iterations depends on the size and variability of the dataset.
4. Computational Cost:
Higher Number of Iterations: More iterations require more computational resources and time. While they provide more accurate and stable estimates, this comes at the cost of increased computational effort, especially for large datasets or complex models.
Balancing Iterations and Resources: It’s essential to balance the number of iterations with available computational resources. In practice, techniques like k-fold cross-validation with k between 5 and 10 are commonly used as a good trade-off between accuracy and computational efficiency.
Conclusion:
Higher iterations generally lead to a better, more accurate estimate of model performance by providing a more stable and reliable measure that accounts for variability in the data. However, beyond a certain point, the benefits of additional iterations diminish, and it becomes important to balance accuracy with computational costs.












Increasing the number of iterations can help mitigate the issues associated with small training or validation datasets, but it has its limitations. Here's how it works and what challenges remain:

1. Mitigating Small Dataset Issues:
Increased Iterations: When dealing with a small dataset, increasing the number of iterations (e.g., using cross-validation with a higher number of folds or repeating the validation process multiple times) allows the model to be trained and validated on different subsets of the data. This exposes the model to more diverse data scenarios, which can help provide a more reliable estimate of performance than a single split would allow.
More Data Utilization: In techniques like leave-one-out cross-validation (LOOCV), where each data point serves as a validation set once while the rest serve as the training set, the model is trained on nearly all the data points available, which is especially valuable when the dataset is small.
2. Challenges and Limitations:
Overfitting Risk: Small datasets inherently carry a higher risk of overfitting. Even with increased iterations, the model may learn noise or specific patterns that do not generalize well to unseen data. This overfitting can still occur despite the more frequent re-sampling, as the underlying dataset size remains limited.
Variance in Estimates: With a small dataset, even with many iterations, the variance in the performance estimates can still be high. This means that the performance metrics might still fluctuate significantly between iterations, making it hard to pin down a truly accurate estimate of test accuracy.
Limited Representativeness: If the small dataset does not capture the full variability of the underlying data distribution, increasing iterations won't fully compensate for the lack of diversity in the data. The model might still perform poorly on unseen test data that includes patterns or features not present in the small training/validation sets.
3. Practical Considerations:
Computational Efficiency: While increasing iterations can help, it also increases computational load. In small datasets, where each data point is precious, using methods like LOOCV or k-fold cross-validation with a high k (e.g., 10 or more) might be appropriate to maximize the use of available data. However, this comes at the cost of significantly more training cycles.
Data Augmentation and Other Techniques: For very small datasets, increasing iterations alone might not be enough. Techniques such as data augmentation (for image or text data), transfer learning, or generating synthetic data could be necessary to artificially increase the effective size of the dataset.
Conclusion:
Increasing iterations can help deal with small train or validation datasets by providing more stable and representative performance estimates. However, it does not fully resolve the limitations inherent to small datasets, such as overfitting and lack of data diversity. To truly address these issues, other techniques, such as data augmentation or transfer learning, might also be needed in conjunction with increased iterations.










